In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn import datasets
from skimage import io
from tqdm import tqdm
import os
import numpy as np


In [ ]:
data_path= '../input/utkface-new/UTKFace'

In [ ]:
from skimage import transform
from skimage import color
def load_utkface_dataset(data_path):
    images = []
    labels_gender = []
    labels_ethnicity = []
    labels_age=[]
    
    for filename in os.listdir(data_path):
        if filename.endswith('.jpg'):
            image_path = os.path.join(data_path, filename)
            try:
                age, gender, ethnicity = map(int, filename.split('_')[:3])
            except ValueError as e:
                print(f"Skipping file {filename} due to error: {e}")
                continue
            
            labels_gender.append(gender)
            labels_ethnicity.append(ethnicity)
            labels_age.append(age)
            img = io.imread(image_path)
            img = transform.resize(img, (128, 128)) 
            img = color.rgb2gray(img)  
            images.append(img)

    return np.array(images), np.array(labels_gender), np.array(labels_ethnicity), np.array(labels_age)

In [ ]:
load_utkface_dataset(data_path)

In [ ]:
images, labels_gender, labels_ethnicity, labels_age = load_utkface_dataset(data_path)
images_flat = images.reshape(images.shape[0], -1)

In [ ]:
labels_age

In [ ]:
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, np.inf]
age_labels = [i for i in range(len(bins) - 1)]

In [ ]:
ages = labels_age
a_l = np.digitize(ages, bins=bins, right=True)
encoded_age_groups = [group - 1 for group in a_l]

In [ ]:
print(encoded_age_groups[8])
labels_age[8]

In [ ]:
pca = PCA(n_components=100)
images_pca = pca.fit_transform(images_flat)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(images_pca, encoded_age_groups, test_size=0.2, random_state=42)

In [ ]:
nb_classifier = GaussianNB()
nb_classifier.fit(X_train, y_train)

In [ ]:
y_pred = nb_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")